In [1]:
import pandas_gbq
import pickle
import pandas as pd
import numpy as np
import re
import time
import os
import math
import random
import warnings

from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import gensim
from nltk.tokenize import word_tokenize
import dateutil.parser

%matplotlib inline

# TODO: Set project_id to your Google Cloud Platform project ID.
# project_id = "xxxxxx-yyyyyy-######"
project_id = "diesel-client-247517"



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jeffb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Pipeline for Stack Overflow Question and Answers Posts Starts Here

In [2]:
# use this to get the table definition of individual tables in Big Query's tables

# substitute the table name you want the schema for in the "table_name = " line below


sql = """

SELECT column_name, data_type
FROM `bigquery-public-data`.stackoverflow.INFORMATION_SCHEMA.COLUMNS  
WHERE table_name = 'posts_questions' 

"""
df=pandas_gbq.read_gbq(sql, project_id=project_id)

In [3]:
df

,column_name,data_type
0,id,INT64
1,title,STRING
2,body,STRING
3,accepted_answer_id,INT64
4,answer_count,INT64
5,comment_count,INT64
6,community_owned_date,TIMESTAMP
7,creation_date,TIMESTAMP
8,favorite_count,INT64
9,last_activity_date,TIMESTAMP


In [15]:
# how many total question and answer records are in Stack Overflow?
sql = """
SELECT COUNT(*) FROM `bigquery-public-data.stackoverflow.posts_questions` 
"""

numquestions = pandas_gbq.read_gbq(sql, project_id=project_id)
print("total number of questions in Stack Overflow:", numquestions)

sql = """
SELECT COUNT(*) FROM `bigquery-public-data.stackoverflow.posts_answers`
"""

numanswers = pandas_gbq.read_gbq(sql, project_id=project_id)
print("total number of answers in Stack Overflow:", numanswers)

total number of questions in Stack Overflow:         f0_
0  18154493
total number of answers in Stack Overflow:         f0_
0  27665009


In [4]:
# Here is the code to extract the records from one table and load it into a pandas data frame

# note that this can run a long time, depending on the number of records you are trying to load


sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
OR tags LIKE '%graph%'
OR tags LIKE '%chart%'
OR tags LIKE '%visualiz%'
OR tags LIKE '%choropleth%'
OR tags LIKE '%drawing%'
OR tags LIKE '%line%'
OR tags LIKE '%geospatial%'
OR tags LIKE '%diagram%')
"""



dfPostQuestionsFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

In [7]:
# write the raw questions dataframe to a tsv
dfPostQuestionsFiltered.to_csv("data/stackoverflow/PostQuestionsFiltered_raw_12_06_19.tsv", sep='\t', index=False)

In [8]:
# in case you want to delete the dfPostQuestionsFiltered dataframe

# ** NOTE** For the rest of the process to work correctly, you must save the raw file after executing the Big Query SQL select above
# (use the cell immediately above to save it), then run this cell to delete the dataframe,
# then re-load it using he cell below with the "keep_default_na" paramter in order to remove the NaN -type values in the dataset

del dfPostQuestionsFiltered

In [10]:
# Use this code to re-load raw Stack Overflow questions to the Pandas df -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to

dfPostQuestionsFiltered = pd.read_csv("data/stackoverflow/PostQuestionsFiltered_raw_12_06_19.tsv", sep='\t', keep_default_na=False)

In [11]:
np.shape(dfPostQuestionsFiltered)

(429665, 20)

In [12]:
dfPostQuestionsFiltered

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,56195999,Apollo client doesn't display an error message,<p>I have configured and set up a fully functi...,56196332.0,1,0,,2019-05-18 04:35:44.290000+00:00,,2019-05-18 05:45:46.330000+00:00,2019-05-18 05:17:59.233000+00:00,,7916354.0,,7916354.0,,1,2,reactjserror-handlinggraphqlapollographql-js,256
1,55971037,Azure Pipeline Gradle build fails for Spring,<p>Im trying to build and deploy a Spring API ...,56004367.0,1,0,,2019-05-03 13:40:43.907000+00:00,,2019-05-06 11:31:53.900000+00:00,,,,,5800859.0,,1,1,javaspringazurespring-bootazure-pipelines,258
2,56028672,Why do I get NA values after filtering a datas...,<p>I am using the gpa1 dataset in R and am try...,,1,0,,2019-05-07 18:38:55.503000+00:00,,2019-05-07 19:19:27.447000+00:00,,,,,11452536.0,,1,0,filteringlinear-regressionresultset,5
3,56224298,Matplotlib could not track button release event,<p>I try to create a dragable line object. The...,,0,0,,2019-05-20 15:46:52.757000+00:00,,2019-05-20 15:53:14.423000+00:00,2019-05-20 15:53:14.423000+00:00,,4089747.0,,4089747.0,,1,0,matplotlib,5
4,56252247,counter of a Dataframe column as a size for sc...,<p>I have used 'Counter' to count the instance...,,0,0,,2019-05-22 08:17:40.557000+00:00,,2019-05-22 08:17:40.557000+00:00,,,,,11232091.0,,1,0,counterscatter-plot,5
5,56281142,Some resources don't enter timesheets in Proje...,<p>I have been asked to enable timesheets in P...,,0,0,,2019-05-23 18:36:05.040000+00:00,,2019-05-23 18:36:05.040000+00:00,,,,,11501306.0,,1,0,projectproject-online,5
6,56212612,Calling a hook (useMutation) inside an event h...,<p>I want to use a mutation in an event handle...,56212940.0,1,0,,2019-05-19 22:47:04.330000+00:00,,2019-05-19 23:57:38.440000+00:00,,,,,10688712.0,,1,0,reactjsgraphqlreact-hooksmutation,261
7,56233232,Graph with O365 Calendar is legacy? What is th...,<p>I want to use an O365 API for synchronisati...,,0,0,,2019-05-21 07:27:44.607000+00:00,,2019-05-21 07:27:44.607000+00:00,,,,,11532180.0,,1,0,graphcalendaroffice365,6
8,56287479,Has somebody working example of TNumericGauge ...,<p>I would like to use TNumericGauge in TDBCha...,,0,0,,2019-05-24 07:08:13.093000+00:00,,2019-05-24 07:08:13.093000+00:00,,,,,11548698.0,,1,0,teechart,6
9,56288322,Process finished with exit code 0 but expectin...,<p>I tried to compute historical value at risk...,,0,0,,2019-05-24 08:05:15.990000+00:00,,2019-05-24 08:05:15.990000+00:00,,,,,11548961.0,,1,0,python-ggplotrisk-analysis,6


In [39]:
# let's delete questions without answers
dfPostQuestionsFiltered['id'][dfPostQuestionsFiltered['answer_count'] < 1].count()

74243

In [40]:
dfPostQuestionsFiltered = dfPostQuestionsFiltered[dfPostQuestionsFiltered['answer_count'] >= 1]

In [41]:
np.shape(dfPostQuestionsFiltered)

(355422, 22)

In [51]:
# let's also delete questions without accepted answers
dfPostQuestionsFiltered['id'][dfPostQuestionsFiltered['accepted_answer_id'] == ''].count()

104835

In [52]:
dfPostQuestionsFiltered = dfPostQuestionsFiltered[dfPostQuestionsFiltered['accepted_answer_id'] != '']

In [53]:
np.shape(dfPostQuestionsFiltered)

(173698, 21)

In [23]:
# read the model tags file to be used to create the tag features from the tagtext field in the questions file, which
# has all tag values concatenated without any separators

# NOTE: Used shortened version of model tag list below

dfModelTags = pd.read_csv("data/stackoverflow/modeltags_shortened.csv")

In [24]:
dfModelTags

,tagtext
0,python
1,scala
2,matlab
3,matplotlib
4,animation
5,d3
6,ggplot2
7,plot
8,graph
9,chart


In [25]:
# add a new column to hold tags from the modeltags table that occur in the queion tags field

dfPostQuestionsFiltered['new_tags']=''

In [28]:
# find occurences of model tag values in tags filed, and append them to tbe new_tags field
def parsetags(qtag):
    tstr=''
    for i in dfModelTags['tagtext']:
        if i in qtag:
            tstr=tstr + i + " "
    return tstr

dfPostQuestionsFiltered['new_tags'] = dfPostQuestionsFiltered['tags'].map(parsetags)


In [29]:
dfPostQuestionsFiltered['new_tags']

0                          graph 
1                                
2                                
3                matplotlib plot 
4                           plot 
5                                
6                          graph 
7                          graph 
8                          chart 
9                    python plot 
10                         graph 
11                               
12                         graph 
13                               
14                               
15                         graph 
16                               
17                               
18                               
19                         graph 
20                         graph 
21                               
22                          plot 
23                         graph 
24                               
25                               
26                               
27                               
28            python plot plotly 
29            

In [22]:
dfPostQuestionsFiltered['tags']

0              reactjserror-handlinggraphqlapollographql-js
1                 javaspringazurespring-bootazure-pipelines
2                       filteringlinear-regressionresultset
3                                                matplotlib
4                                       counterscatter-plot
5                                     projectproject-online
6                         reactjsgraphqlreact-hooksmutation
7                                    graphcalendaroffice365
8                                                  teechart
9                                python-ggplotrisk-analysis
10          javascriptreactjsgraphqlreact-apolloreact-hooks
11        sharepointconnectiontimeoutsharepoint-onlinere...
12                      microsoft-graphmicrosoft-graph-mail
13        githubcommand-line-interfacerebasebranching-an...
14                    springcommand-linespring-cloud-config
15                                    graphundirected-graph
16                                  webc

In [32]:
# Here is a separate process to check for references to the r language in the question body
# and set a separate 'r' column in the questions df to be used to update new_tags
#
# we cannot use the same process used on the tags above, since it will almost always find occurrences of 'r'
# in the tags field that are just parts of other terms (for example, the "r" in "chart")
#
# So, here we look for standalone references to r in the question body instead
#
dfPostQuestionsFiltered['r'] = ''

def testforr(x):
    if (' r ' in x) or (' r,' in x) or (' r.' in x) or (' r:' in x):
        return 'r '
    else:
        return ''

dfPostQuestionsFiltered['r'] = dfPostQuestionsFiltered['body'].map(testforr)


In [33]:
# now, update new_tags, appending the 'r' for questions referencing r in the question body
dfPostQuestionsFiltered['new_tags'] = dfPostQuestionsFiltered['new_tags'] + dfPostQuestionsFiltered['r']

In [36]:
dfPostQuestionsFiltered['new_tags'][dfPostQuestionsFiltered['r']=='r ']

202                     d3 graph r 
243       python matplotlib plot r 
257                              r 
273                              r 
308                         plot r 
368           python plot plotly r 
429                        graph r 
434                 ggplot2 plot r 
475                              r 
593                        chart r 
675       python matplotlib plot r 
740                 ggplot2 plot r 
751                 python graph r 
756                        chart r 
859       python matplotlib plot r 
870                 ggplot2 plot r 
894                 python graph r 
1014            d3 visualization r 
1092      python matplotlib plot r 
1233                 plot plotly r 
1236                 matlab plot r 
1347                ggplot2 plot r 
1496                python graph r 
1523         ggplot2 plot plotly r 
1577                ggplot2 plot r 
1613                       graph r 
1683                ggplot2 plot r 
1738                       g

In [42]:
# drop the 'r' column, as it is no longer needed
del dfPostQuestionsFiltered['r']

In [43]:
dfPostQuestionsFiltered.loc[0]

id                                                                   56195999
title                          Apollo client doesn't display an error message
body                        <p>I have configured and set up a fully functi...
accepted_answer_id                                                 56196332.0
answer_count                                                                1
comment_count                                                               0
community_owned_date                                                         
creation_date                                2019-05-18 04:35:44.290000+00:00
favorite_count                                                               
last_activity_date                           2019-05-18 05:45:46.330000+00:00
last_edit_date                               2019-05-18 05:17:59.233000+00:00
last_editor_display_name                                                     
last_editor_user_id                                             

In [44]:
# now, let's drop any questions that do not have tags we are interested in

dfPostQuestionsFiltered['id'][dfPostQuestionsFiltered['new_tags'] == ''].count()

76889

In [45]:
dfPostQuestionsFiltered = dfPostQuestionsFiltered[dfPostQuestionsFiltered['new_tags'] != '']

In [46]:
np.shape(dfPostQuestionsFiltered)

(278533, 21)

In [54]:
# let's now create a separate question body column, 'cleaned_question_body', that removes all the code and all HTL tags
# from the question body
# We might use the cleaned_question_body as a feature in one of the similarity models
def clean_ques_body_text(body):
    tmp_body=BeautifulSoup(body,"lxml")
    for code in tmp_body('code'):
        code.replace_with('')
    tstr=''
    for string in tmp_body.stripped_strings:
        tstr = tstr + ' ' + string
    return tstr

dfPostQuestionsFiltered['cleaned_question_body']=''

sttime=time.time()    
dfPostQuestionsFiltered['cleaned_question_body'] = dfPostQuestionsFiltered['body'].map(clean_ques_body_text)
print('question body cleaned in',(time.time()-sttime)/60,'minutes')    

C:\Users\jeffb\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\jeffb\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


question body cleaned in 1.1601197679837545 minutes


In [55]:
dfPostQuestionsFiltered['body'].head()

0     <p>I have configured and set up a fully functi...
6     <p>I want to use a mutation in an event handle...
10    <p>I have useQuery and useMutation from react-...
70    <p>I'm coding in R and I have the following da...
72    <p>I am graphing the internet connection in my...
Name: body, dtype: object

In [56]:
dfPostQuestionsFiltered['cleaned_question_body'].head()

0      I have configured and set up a fully function...
6      I want to use a mutation in an event handler....
10     I have useQuery and useMutation from react-ap...
70     I'm coding in R and I have the following data...
72     I am graphing the internet connection in my a...
Name: cleaned_question_body, dtype: object

In [59]:
print('questions with cleaned_question_body == Nan',len(dfPostQuestionsFiltered['cleaned_question_body'][dfPostQuestionsFiltered['cleaned_question_body'].isna()]))

questions with cleaned_question_body == Nan 0


In [60]:
print('questions with cleaned_question_body == "" ',len(dfPostQuestionsFiltered['cleaned_question_body'][dfPostQuestionsFiltered['cleaned_question_body'] == '']))

questions with cleaned_question_body == ""  21


In [100]:
# write the dataframe to a tsv
dfPostQuestionsFiltered.to_csv("data/stackoverflow/PostQuestionsFiltered_V5_parsed.tsv", sep='\t', index=False)

In [101]:
# write the dataframe to a csv
dfPostQuestionsFiltered.to_csv("data/stackoverflow/PostQuestionsFiltered_V5_parsed.csv",index=False)

In [2]:
# USE THIS CODE TO LOAD THE STACK OVERFLOW POST QUESTIONS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfPostQuestionsFiltered = pd.read_csv("data/stackoverflow/PostQuestionsFiltered_V5_parsed.tsv", sep='\t', keep_default_na=False)

In [3]:
dfPostQuestionsFiltered

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,...,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count,new_tags,cleaned_question_body
0,56195999,Apollo client doesn't display an error message,<p>I have configured and set up a fully functi...,56196332.0,1,0,,2019-05-18 04:35:44.290000+00:00,,2019-05-18 05:45:46.330000+00:00,...,7916354.0,,7916354.0,,1,2,reactjserror-handlinggraphqlapollographql-js,256,graph,I have configured and set up a fully function...
1,56212612,Calling a hook (useMutation) inside an event h...,<p>I want to use a mutation in an event handle...,56212940.0,1,0,,2019-05-19 22:47:04.330000+00:00,,2019-05-19 23:57:38.440000+00:00,...,,,10688712.0,,1,0,reactjsgraphqlreact-hooksmutation,261,graph,I want to use a mutation in an event handler....
2,56204854,How do I chain useQuery and useMutation in Gra...,<p>I have useQuery and useMutation from react-...,56206915.0,2,0,,2019-05-19 04:52:19.730000+00:00,,2019-07-15 12:45:21.457000+00:00,...,,,10116367.0,,1,0,javascriptreactjsgraphqlreact-apolloreact-hooks,262,graph,I have useQuery and useMutation from react-ap...
3,56189061,Customize lines based on specific criteria wit...,<p>I'm coding in R and I have the following da...,56197449.0,1,0,,2019-05-17 15:01:14.930000+00:00,,2019-05-18 08:58:44.763000+00:00,...,,,9537627.0,,1,0,rplotly,11,plot plotly,I'm coding in R and I have the following data...
4,56203260,How do I sum a list with its previous entries ...,<p>I am graphing the internet connection in my...,56203343.0,1,0,,2019-05-18 22:17:19.290000+00:00,,2019-05-18 22:32:10.980000+00:00,...,,,8598201.0,,1,0,pythonlistgraphsumiteration,11,python graph,I am graphing the internet connection in my a...
5,56218250,Modify y data in a line,<p>I have the following script</p>\r\r\n\r\r\n...,56220931.0,1,0,,2019-05-20 09:43:44.660000+00:00,,2019-05-20 12:22:29.497000+00:00,...,,,2749397.0,,1,0,matplotlib,11,matplotlib plot,I have the following script Expected Behavior...
6,56275333,Display Google Geochart in log scale,"<p>I'm using Google's <a href=""https://develop...",56275655.0,1,0,,2019-05-23 12:33:33.063000+00:00,,2019-05-23 12:51:20.197000+00:00,...,,,930151.0,,1,0,chartsgoogle-visualizationgeo,11,chart visualization,I'm using Google's geochart . I'm using the n...
7,55972829,Data displayed upon submission of a form is vi...,<p>I am trying to create create a calculator(f...,55973074.0,1,0,,2019-05-03 15:22:50.823000+00:00,,2019-05-03 15:36:57.783000+00:00,...,,,10067658.0,,1,0,javascriptplotly.js,12,plot plotly,I am trying to create create a calculator(for...
8,56095434,Every row given must be either null or an arra...,<p>I am trying to draw a line from google char...,56095454.0,1,0,,2019-05-12 01:01:47.690000+00:00,,2019-05-12 01:16:52.103000+00:00,...,,,11375519.0,,1,0,phpmysqlgoogle-visualization,12,visualization,I am trying to draw a line from google chart ...
9,56260378,Make Highcharts pick a smart y-axis min,"<p>For a bar or column chart in Highcharts, is...",56260575.0,1,0,,2019-05-22 15:33:03.250000+00:00,,2019-05-22 15:44:21.623000+00:00,...,,,614263.0,,1,0,chartshighchartsstatistics,13,chart highcharts,"For a bar or column chart in Highcharts, is t..."


In [61]:
# retrieving the answers that belong to the questions retrieved above

sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.posts_answers` WHERE parent_id IN 
    (SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
        OR tags LIKE '%graph%'
        OR tags LIKE '%chart%'
        OR tags LIKE '%visualiz%'
        OR tags LIKE '%choropleth%'
        OR tags LIKE '%drawing%'
        OR tags LIKE '%line%'
        OR tags LIKE '%geospatial%'
        OR tags LIKE '%diagram%'))
"""
dfPostAnswersFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

In [62]:
np.shape(dfPostAnswersFiltered)

(545457, 20)

In [ ]:
# write the dataframe to a tsv
dfPostAnswersFiltered.to_csv("data/stackoverflow/PostAnswersFiltered_raw_12_06_19.tsv", sep='\t', index=False)

In [ ]:
# in case you want to delete the dfPostQuestionsFiltered dataframe

# ** NOTE** For the rest of the process to work correctly, you must save the raw file after executing the Big Query SQL select above
# (use the cell immediately above to save it), then run this cell to delete the dataframe,
# then re-load it using he cell below with the "keep_default_na" paramter in order to remove the NaN -type values in the dataset

del dfPostAnswersFiltered

In [67]:
# Use this code to re-load raw Stack Overflow questions to the Pandas df -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to

dfPostAnswersFiltered = pd.read_csv("data/stackoverflow/PostAnswersFiltered_raw_12_06_19.tsv", sep='\t', keep_default_na=False)

In [69]:
np.shape(dfPostAnswersFiltered)

(545457, 20)

In [71]:
dfPostAnswersFiltered

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,44629055,,<p>One possible solution: I do not actually n...,,,0,,2017-06-19 11:12:02.083000+00:00,,2017-06-19 11:12:02.083000+00:00,,,,,8138226.0,44567221,2,0,,
1,44629646,,"<p>I've changed your function in a little, try...",,,0,,2017-06-19 11:43:10.483000+00:00,,2017-06-19 11:43:10.483000+00:00,,,,,6618051.0,41982761,2,0,,
2,44630291,,<p>Currently there is no click event on catego...,,,0,,2017-06-19 12:13:24.530000+00:00,,2017-06-19 12:13:24.530000+00:00,,,,,3437934.0,44297012,2,0,,
3,44631329,,<p>It's better to create separate files for CS...,,,0,,2017-06-19 13:01:51+00:00,,2017-06-19 13:01:51+00:00,,,,,3582474.0,44630985,2,0,,
4,44632276,,<p>The correct syntax is this one:</p>\r\n\r\n...,,,0,,2017-06-19 13:43:45.580000+00:00,,2017-06-19 13:43:45.580000+00:00,,,,,5222773.0,43895941,2,0,,
5,44632434,,<p>You can define the lines from the dataset w...,,,0,,2017-06-19 13:51:27.790000+00:00,,2017-06-19 13:51:27.790000+00:00,,,,,3951036.0,44621561,2,0,,
6,44633649,,<p>The way matplotlib is working is that you h...,,,0,,2017-06-19 14:47:46.830000+00:00,,2017-06-19 14:47:46.830000+00:00,,,,,4124317.0,44633422,2,0,,
7,44633675,,"<p>I have figured out the answer, which is to ...",,,0,,2017-06-19 14:48:42.900000+00:00,,2017-06-19 14:48:42.900000+00:00,,,,,8050923.0,44624707,2,0,,
8,44634042,,<p>I resolved it.</p>\r\n\r\n<pre><code>Get-Co...,,,0,,2017-06-19 15:06:24.867000+00:00,,2017-06-19 15:06:24.867000+00:00,,,,,7065080.0,44633338,2,0,,
9,44635048,,<p>To further a_guest answer - To enable see t...,,,0,,2017-06-19 15:56:06.457000+00:00,,2017-06-19 15:56:06.457000+00:00,,,,,1704227.0,44632018,2,0,,


In [74]:
# let's drop any answers that do not have corresponding questions in our questions dataframe
tmpid=pd.DataFrame(dfPostQuestionsFiltered['id']) # temporary dataframe to hold ids from questions for merge operation
tmpid.columns=['tempqid']
dfPostAnswersFiltered=pd.merge(dfPostAnswersFiltered, tmpid, how='inner',left_on='parent_id',right_on='tempqid')
del dfPostAnswersFiltered['tempqid']
del tmpid

Index(['id', 'title', 'body', 'accepted_answer_id', 'answer_count',
       'comment_count', 'community_owned_date', 'creation_date',
       'favorite_count', 'last_activity_date', 'last_edit_date',
       'last_editor_display_name', 'last_editor_user_id', 'owner_display_name',
       'owner_user_id', 'parent_id', 'post_type_id', 'score', 'tags',
       'view_count'],
      dtype='object')

In [86]:
np.shape(dfPostAnswersFiltered)

(261052, 21)

In [89]:
# check the answer body of each answer and extract the images references into a separate column as a list
def extract_image_references(body):
    tmp_body=BeautifulSoup(body,"lxml")
    timgs=[]
    for img in tmp_body('img'):
        timgs.append(img)
    return timgs

sttime=time.time()    

dfPostAnswersFiltered['images_list']=''

dfPostAnswersFiltered['images_list'] = dfPostAnswersFiltered['body'].map(extract_image_references)
print('images finished in',(time.time()-sttime)/60,'minutes')    

images finished in 4.315905344486237 minutes


In [90]:
# check the answer body of each answer and extract the code snippets into a separate column as a list
def extract_code_snippets(body):
    tmp_body=BeautifulSoup(body,"lxml")
    tcode_snips=[]
    for code in tmp_body('code'):
        tcode_snips.append(code)
    return tcode_snips

sttime=time.time()    

dfPostAnswersFiltered['code_snippets']=''

dfPostAnswersFiltered['code_snippets'] = dfPostAnswersFiltered['body'].map(extract_code_snippets)
print('code snippets finished in',(time.time()-sttime)/60,'minutes')    

code snippets finished in 4.729599332809448 minutes


In [91]:
# create a separate 'cleaned_body' column from the answer 'body' column with all code and all html tags removed
def clean_body_text(body):
    tmp_body=BeautifulSoup(body,"lxml")
    for code in tmp_body('code'):
        code.replace_with('')
    tstr=''
    for string in tmp_body.stripped_strings:
        tstr = tstr + ' ' + string
    return tstr

dfPostAnswersFiltered['cleaned_body']=''

sttime=time.time()    
dfPostAnswersFiltered['cleaned_body'] = dfPostAnswersFiltered['body'].map(clean_body_text)
print('body cleaned in',(time.time()-sttime)/60,'minutes')  

body cleaned in 4.306883104642233 minutes


In [92]:
dfPostAnswersFiltered['body'].head()

0    <p>The correct syntax is this one:</p>\r\n\r\n...
1    <p>You can download the plugin manually from t...
2    <p>You can define the lines from the dataset w...
3    <p>The way matplotlib is working is that you h...
4    <p>I have figured out the answer, which is to ...
Name: body, dtype: object

In [93]:
dfPostAnswersFiltered['cleaned_body'].head()

0     The correct syntax is this one: It is working...
1     You can download the plugin manually from the...
2     You can define the lines from the dataset wit...
3     The way matplotlib is working is that you hav...
4     I have figured out the answer, which is to us...
Name: cleaned_body, dtype: object

In [98]:
dfPostAnswersFiltered['images_list'].head()

0    []
1    []
2    []
3    []
4    []
Name: images_list, dtype: object

In [95]:
dfPostAnswersFiltered['code_snippets'].head()

0    [<code>plugin install com.graphaware.es/graph-...
1                                                   []
2           [<code>mapping.get(i, "anchor"),
</code>]
3    [<code>def error_plot(ax, title, x_data, y_dat...
4    [<code>%sql select t1.time, t1.value,coalesce(...
Name: code_snippets, dtype: object

In [107]:
# add an image count column to the answers so that we can filter on answers with images
def countimages(image_list):
    if image_list == []:
        return 0
    else:
        return len(image_list)
    
dfPostAnswersFiltered['images_count']=0
dfPostAnswersFiltered['images_count'] = dfPostAnswersFiltered['images_list'].map(countimages)

In [108]:
dfPostAnswersFiltered['images_count'][dfPostAnswersFiltered['images_count'] > 0].head()

12    3
29    1
32    1
35    1
37    3
Name: images_count, dtype: int64

In [109]:
dfPostAnswersFiltered['id'][dfPostAnswersFiltered['images_count'] > 0].count()

45991

In [110]:
# write the dataframe to a tsv
dfPostAnswersFiltered.to_csv("data/stackoverflow/PostAnswersFiltered_V5_parsed.tsv", sep='\t', index=False)

In [4]:
# USE THIS CODE TO LOAD THE STACK OVERFLOW POST ANSWERS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfPostAnswersFiltered = pd.read_csv("data/stackoverflow/PostAnswersFiltered_V5_parsed.tsv", sep='\t', keep_default_na=False)

In [202]:
dfPostAnswersFiltered

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,...,owner_user_id,parent_id,post_type_id,score,tags,view_count,images_list,code_snippets,cleaned_body,images_count
0,44632276,,<p>The correct syntax is this one:</p>\r\r\n\r...,,,0,,2017-06-19 13:43:45.580000+00:00,,2017-06-19 13:43:45.580000+00:00,...,5222773.0,43895941,2,0,,,[],[<code>plugin install com.graphaware.es/graph-...,The correct syntax is this one: It is working...,0
1,43934125,,<p>You can download the plugin manually from t...,,,13,,2017-05-12 09:28:09.857000+00:00,,2017-05-12 09:28:09.857000+00:00,...,2662355.0,43895941,2,1,,,[],[],You can download the plugin manually from the...,0
2,44632434,,<p>You can define the lines from the dataset w...,,,0,,2017-06-19 13:51:27.790000+00:00,,2017-06-19 13:51:27.790000+00:00,...,3951036.0,44621561,2,0,,,[],"[<code>mapping.get(i, ""anchor""),\r\r\n</code>]",You can define the lines from the dataset wit...,0
3,44633649,,<p>The way matplotlib is working is that you h...,,,0,,2017-06-19 14:47:46.830000+00:00,,2017-06-19 14:47:46.830000+00:00,...,4124317.0,44633422,2,0,,,[],"[<code>def error_plot(ax, title, x_data, y_dat...",The way matplotlib is working is that you hav...,0
4,44633675,,"<p>I have figured out the answer, which is to ...",,,0,,2017-06-19 14:48:42.900000+00:00,,2017-06-19 14:48:42.900000+00:00,...,8050923.0,44624707,2,0,,,[],"[<code>%sql select t1.time, t1.value,coalesce(...","I have figured out the answer, which is to us...",0
5,44635048,,<p>To further a_guest answer - To enable see t...,,,0,,2017-06-19 15:56:06.457000+00:00,,2017-06-19 15:56:06.457000+00:00,...,1704227.0,44632018,2,0,,,[],[],To further a_guest answer - To enable see the...,0
6,44632901,,<p>The problem is that the Python process fini...,,,3,,2017-06-19 14:12:13.690000+00:00,,2017-06-19 14:12:13.690000+00:00,...,3767239.0,44632018,2,2,,,[],"[<code>for</code>, <code>-i</code>, <code>pyth...",The problem is that the Python process finish...,0
7,49985313,,<p>You can keep the window open by creating a ...,,,0,,2018-04-23 15:59:00.010000+00:00,,2018-04-23 15:59:00.010000+00:00,...,6605826.0,44632018,2,1,,,[],"[<code>QApplication</code>, <code>exec_()</cod...",You can keep the window open by creating a at...,0
8,44636285,,<p>Since you've mentioned that the transformat...,,,0,,2017-06-19 17:10:08.570000+00:00,,2017-06-19 17:10:08.570000+00:00,...,8060278.0,18506201,2,0,,,[],[<code>lambda=3.79\r\r\nm2=lm(resp^((lambda-1)...,Since you've mentioned that the transformatio...,0
9,18507004,,"<pre><code>plotexpl &lt;- seq(1,4,length.out=1...",,,3,,2013-08-29 09:24:47.800000+00:00,,2013-08-29 09:24:47.800000+00:00,...,1412059.0,18506201,2,2,,,[],"[<code>plotexpl &lt;- seq(1,4,length.out=10)\r...",I won't discuss the statistical issues here (...,0


In [13]:
# retrieving user ids for owners of the answers retrieved above

sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.users` WHERE id IN
    (SELECT owner_user_id FROM `bigquery-public-data.stackoverflow.posts_answers` WHERE parent_id IN 
        (SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
            OR tags LIKE '%graph%'
            OR tags LIKE '%chart%'
            OR tags LIKE '%visualiz%'
            OR tags LIKE '%choropleth%'
            OR tags LIKE '%drawing%'
            OR tags LIKE '%line%'
            OR tags LIKE '%geospatial%'
            OR tags LIKE '%diagram%')))
"""
dfUsersFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

In [49]:
np.shape(dfUsersFiltered)

(304366, 13)

In [50]:
dfUsersFiltered

,id,display_name,about_me,age,creation_date,last_access_date,location,reputation,up_votes,down_votes,views,profile_image_url,website_url
0,1228,Will,<p>Downvoted a post? Want to let the author kn...,None,2008-08-13 13:58:55.613000+00:00,2019-08-30 19:32:14.180000+00:00,United States,114479,4072,8499,61068,None,https://idownvotedbecau.se
1,2009,hometoast,<p>(your about me is currently blank)</p>,None,2008-08-19 19:16:03.210000+00:00,2019-08-29 14:39:17.043000+00:00,"Mountain Top, PA",9604,1400,72,892,None,http://devtoast.com
2,2684,Martin Marconcini,<p>I do Android development with Kotlin/Java a...,None,2008-08-24 14:38:27.090000+00:00,2019-08-31 20:36:33.830000+00:00,"Almere, NL",17343,2555,159,5171,https://i.stack.imgur.com/Z9oZ0.jpg?s=128&g=1,https://professorneurus.wordpress.com
3,5049,macbirdie,"<p>Freelancer, doing primarily iOS development...",None,2008-09-07 15:03:23.483000+00:00,2019-08-27 14:14:57.663000+00:00,"Poznan, Poland",14626,344,49,3603,None,https://macbirdie.net
4,6651,Alex Angas,None,None,2008-09-15 12:25:40.173000+00:00,2019-08-30 16:49:12.890000+00:00,"London, United Kingdom",35087,3339,271,3365,None,https://alexangas.com
5,8562,mdb,None,None,2008-09-15 16:20:52.917000+00:00,2017-11-14 13:45:05.013000+00:00,Netherlands,45600,1084,65,1841,None,None
6,8912,Ted Naleid,None,None,2008-09-15 17:12:19.533000+00:00,2019-08-29 13:28:19.343000+00:00,United States,22733,1251,26,1292,None,http://naleid.com/
7,9936,Martin Cote,<p>Graphics Engineer at Unity3D.</p>,None,2008-09-15 20:30:09.543000+00:00,2019-08-23 18:37:35.430000+00:00,"Montreal, Canada",22119,316,72,1305,None,None
8,13820,Sridhar Iyer,Wired and Weird,None,2008-09-16 20:57:40.933000+00:00,2019-08-30 19:03:38.590000+00:00,"Santa Clara, CA",2021,176,22,687,None,http://sridharv.net
9,17413,Nifle,<p><code>A rather lazy sysadmin and sometimes ...,None,2008-09-18 08:34:03.827000+00:00,2019-08-30 12:50:59.610000+00:00,Sweden,8624,2964,422,1470,None,http://www.kiva.org/


In [51]:
# write the dataframe to a tsv
dfUsersFiltered.to_csv("data/stackoverflow/UsersFiltered_V3.tsv", sep='\t')

In [4]:
# USE THIS CODE TO LOAD THE STACK OVERFLOW USERS FOR ABOVE ANSWERS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfUsersFiltered = pd.read_csv("data/stackoverflow/UsersFiltered_V3.tsv", sep='\t')

In [15]:
# retrieving the comments that belong to the questions retrieved above

# NOTE - the below select may only retrieve comments related to questions??

# is a separate select needed to retrieve comments related to the answers?

sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.comments` WHERE post_id IN 
    (SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
        OR tags LIKE '%graph%'
        OR tags LIKE '%chart%'
        OR tags LIKE '%visualiz%'
        OR tags LIKE '%choropleth%'
        OR tags LIKE '%drawing%'
        OR tags LIKE '%line%'
        OR tags LIKE '%geospatial%'
        OR tags LIKE '%diagram%'))
"""
dfCommentsFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

In [16]:
np.shape(dfCommentsFiltered)

(1184056, 7)

In [17]:
dfCommentsFiltered

,id,text,creation_date,post_id,user_id,user_display_name,score
0,41452359,SOLVED! Check this https://github.com/ikimuhen...,2014-10-16 09:50:36.890000+00:00,26350205,1833505.0,None,4
1,41453409,You also get this problem when you restore a b...,2014-10-16 10:23:14.340000+00:00,2043382,10245.0,None,4
2,41464240,"Do you think `-12.80010128657071, 1121.2874782...",2014-10-16 15:09:57.510000+00:00,22773651,1238965.0,None,4
3,41472601,Pretty much all encryption algorithms are goin...,2014-10-16 19:27:46.807000+00:00,26412089,869736.0,None,4
4,41473301,Surely F(S) = 9999 is not what you're looking ...,2014-10-16 19:48:05.153000+00:00,26412312,2144669.0,None,4
5,41474727,"""We are not allowed to do any changes with res...",2014-10-16 20:32:31.033000+00:00,26413343,1274820.0,None,4
6,41490904,I think that kind of question is better to ask...,2014-10-17 10:13:03.323000+00:00,26419181,1451635.0,None,4
7,41499177,The jqChart plugin isn't loaded... check your ...,2014-10-17 14:23:53.250000+00:00,26427123,1686485.0,None,5
8,41503605,Hmmm... it looks like you are cancelling the e...,2014-10-17 16:22:45.360000+00:00,26429055,2718864.0,None,5
9,41568509,This question appears to be off-topic because ...,2014-10-20 12:06:44.173000+00:00,26462355,47961.0,None,4


In [18]:
# write the dataframe to a tsv
dfCommentsFiltered.to_csv("data/stackoverflow/CommentsFiltered_v3.tsv", sep='\t')

In [41]:
# USE THIS CODE TO LOAD THE STACK OVERFLOW POST COMMENTS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfCommentsFiltered = pd.read_csv("data/stackoverflow/CommentsFiltered.tsv", sep='\t')

In [42]:
dfCommentsFiltered

,Unnamed: 0,id,text,creation_date,post_id,user_id,user_display_name,score
0,0,13959910,Don't worry about extra objects unless there i...,2012-05-24 19:03:24.103000+00:00,10743622,NaN,user166390,6
1,1,14116297,"If you're going to use NodeJS, you might as we...",2012-05-31 20:28:18.247000+00:00,10840689,NaN,user1106925,5
2,2,14834466,"WTF, you want to write your own virtual filesy...",2012-07-01 04:15:00.443000+00:00,11279477,NaN,user405725,6
3,3,13253544,"Sorry about the semicolons, I did put them in ...",2012-04-24 13:46:43.277000+00:00,10298983,NaN,user818700,0
4,4,13254271,"@Pointy - I'm so sorry, it's obviously too lat...",2012-04-24 14:11:36.220000+00:00,10298983,NaN,user818700,0
5,5,13255022,Ahhhh.. Now I see. Thank you very much. I real...,2012-04-24 14:37:07.523000+00:00,10298983,NaN,user818700,0
6,6,13256213,Here's a complete example: http://codepad.org/...,2012-04-24 15:18:30.447000+00:00,10290177,NaN,user393964,0
7,7,13279186,"I'm certainly surprised by this, and it does't...",2012-04-25 12:52:39.417000+00:00,10315746,NaN,user24359,0
8,8,13279391,"Er, I mean `Payments`",2012-04-25 12:59:19.757000+00:00,10315746,NaN,user24359,0
9,9,13283132,@dasblinkenlight great thanks. I guess this is...,2012-04-25 15:09:25.203000+00:00,10318302,NaN,user517491,0


### Pipeline for Stack Exchange Data Science Community Question and Answers Posts Starts Here

In [115]:
# Use this code to load raw Data Science Stack Exchange questions and answers to Pandas dfs -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to

dfDSPosts = pd.read_csv("Data/Stack Exchange Data Science/Posts_From_Data_Science_Stack_Exchange.csv", keep_default_na=False)

dfDSQues=dfDSPosts[dfDSPosts['PostTypeId']==1]
dfDSAns=dfDSPosts[dfDSPosts['PostTypeId']==2]

del dfDSPosts

In [120]:
np.shape(dfDSQues)

(20502, 22)

In [121]:
np.shape(dfDSAns)

(22666, 22)

In [122]:
dfDSQues.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
2,54306,1,,,2019-06-22 19:34:13,,0,548,<p>I am working on a project that uses object ...,73678,...,,2019-06-24 04:33:04,2019-06-24 04:33:04,Transfer learning on yolo using keras,<deep-learning><keras><cnn><object-detection><...,0,1,,,
3,54307,1,,,2019-06-22 20:22:13,,0,32,<p>I'm trying to develop a neural network mode...,76495,...,,2019-06-22 20:50:13,2019-06-22 20:50:13,loss function for model that predicts a matrix,<machine-learning><keras><game>,0,0,,,
4,54313,1,54326,,2019-06-23 01:25:13,,0,126,"<p><a href=""https://github.com/Hyeokreal/Actor...",69646,...,,2019-06-25 14:16:13,2019-06-25 14:16:13,A2C Continuous for Pendulum-v0 working impleme...,<neural-network><distribution><gaussian><opena...,1,0,,,
6,54315,1,,,2019-06-23 02:44:03,,0,97,<p>I just resized the image dataset with Pillo...,76386,...,,2019-06-23 05:43:38,2019-06-23 05:43:38,Image is in JPEG but Torchvision shows image e...,<pytorch><image-preprocessing>,0,0,,,
7,54316,1,,,2019-06-23 03:44:17,,0,12,<p>I've trained and developed a Haar Cascade c...,75423,...,,,2019-06-23 03:44:17,Can Haar cascades be used for object tracking ...,<machine-learning><object-detection>,0,0,,,


In [123]:
dfDSAns.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,54303,2,,54237,2019-06-22 18:41:15,,1,,<p>You say you use linear regression. There ar...,71442,...,,,2019-06-22 18:41:15,,,,0,,,
1,54304,2,,54293,2019-06-22 19:06:46,,0,,<p>There are many points that you should check...,71675,...,,2019-06-23 16:04:51,2019-06-23 16:04:51,,,,5,,,
5,54314,2,,54281,2019-06-23 01:49:40,,1,,"<p>First, the remark from the wikipedia articl...",55122,...,,,2019-06-23 01:49:40,,,,5,,,
10,54319,2,,53478,2019-06-23 05:23:02,,1,,<p>As We should not remove any data ... we can...,76129,...,,,2019-06-23 05:23:02,,,,0,,,
14,54323,2,,54263,2019-06-23 09:18:46,,1,,"<p>We have </p>\n\n<p><span class=""math-contai...",38892,...,,,2019-06-23 09:18:46,,,,0,,,


In [134]:
# let's delete questions without answers 
dfDSQues['AnswerCount'][dfDSQues['AnswerCount'] != '0'].count()

14879

In [135]:
dfDSQues = dfDSQues[dfDSQues['AnswerCount'] != '0' ]

In [136]:
np.shape(dfDSQues)

(14879, 22)

In [139]:
# let's also delete questions without accepted answers
dfDSQues['Id'][dfDSQues['AcceptedAnswerId'] == ''].count()

8185

In [140]:
dfDSQues = dfDSQues[dfDSQues['AcceptedAnswerId'] != '']

In [141]:
np.shape(dfDSQues)

(6694, 22)

In [142]:
# read the model tags file to be used to create the tag features from the tagtext field in the questions file, which
# has all tag values concatenated without any separators

# NOTE: Used shortened version of model tag list below

dfModelTags = pd.read_csv("data/stackoverflow/modeltags_shortened.csv")

In [143]:
dfModelTags

,tagtext
0,python
1,scala
2,matlab
3,matplotlib
4,animation
5,d3
6,ggplot2
7,plot
8,graph
9,chart


In [144]:
# add a new column to hold tags from the modeltags table that occur in the queion tags field

dfDSQues['new_tags']=''

C:\Users\jeffb\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [145]:
# find occurences of model tag values in tags filed, and append them to tbe new_tags field
def parsetags(qtag):
    tstr=''
    for i in dfModelTags['tagtext']:
        if i in qtag:
            tstr=tstr + i + " "
    return tstr

dfDSQues['new_tags'] = dfDSQues['Tags'].map(parsetags)


C:\Users\jeffb\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [146]:
dfDSQues['new_tags']

4                      
17                     
24                     
27                     
30                     
49                     
53              python 
56              python 
58                     
61                     
63                     
65              python 
69                     
84       visualization 
86                     
90              python 
101                    
103                    
109             python 
114                    
117                    
119                    
122              scala 
123                    
125                    
134                    
138                    
140                    
149                    
155                    
              ...      
43483                  
43484           python 
43488                  
43489                  
43490                  
43493                  
43501                  
43506           python 
43512                  
43516                  
43518           

In [148]:
dfDSQues['Tags']

4        <neural-network><distribution><gaussian><opena...
17         <deep-learning><training><recurrent-neural-net>
24                                 <machine-learning><svm>
27                                      <r><data-cleaning>
30                         <machine-learning><probability>
49       <machine-learning><deep-learning><supervised-l...
53                            <python><time-series><excel>
56       <machine-learning><python><data-mining><numpy>...
58           <deep-learning><keras><loss-function><metric>
61                                  <dataset><csv><matrix>
63       <machine-learning><categorical-data><beginner>...
65          <python><neural-network><deep-learning><keras>
69                                              <features>
84       <visualization><machine-learning-model><data-s...
86       <machine-learning><neural-network><data-scienc...
90       <machine-learning><python><data-mining><numpy>...
101                                   <data-cleaning><sq

In [150]:
# Here is a separate process to check for references to the r language in the question body
# and set a separate 'r' column in the questions df to be used to update new_tags
#
# we cannot use the same process used on the tags above, since it will almost always find occurrences of 'r'
# in the tags field that are just parts of other terms (for example, the "r" in "chart")
#
# So, here we look for standalone references to r in the question body instead
#
dfDSQues['r'] = ''

def testforr(x):
    if (' r ' in x) or (' r,' in x) or (' r.' in x) or (' r:' in x):
        return 'r '
    else:
        return ''

dfDSQues['r'] = dfDSQues['Body'].map(testforr)


In [151]:
# now, update new_tags, appending the 'r' for questions referencing r in the question body
dfDSQues['new_tags'] = dfDSQues['new_tags'] + dfDSQues['r']

In [152]:
dfDSQues['new_tags'][dfDSQues['r']=='r ']

2172     python r 
2390            r 
3188            r 
3681     python r 
3827            r 
5035            r 
6090     python r 
6326            r 
6397            r 
7327            r 
8033            r 
8474            r 
10174           r 
10562           r 
10815    python r 
11947           r 
12010           r 
15514    python r 
15971           r 
16782           r 
20475    python r 
21000    python r 
25241           r 
26324           r 
27178           r 
29892           r 
30533           r 
33416           r 
34077           r 
37844           r 
41343           r 
41706           r 
41891           r 
42824    python r 
Name: new_tags, dtype: object

In [153]:
# drop the 'r' column, as it is no longer needed
del dfDSQues['r']

In [155]:
dfDSQues.iloc[0]

Id                                                                   54313
PostTypeId                                                               1
AcceptedAnswerId                                                     54326
ParentId                                                                  
CreationDate                                           2019-06-23 01:25:13
DeletionDate                                                              
Score                                                                    0
ViewCount                                                              126
Body                     <p><a href="https://github.com/Hyeokreal/Actor...
OwnerUserId                                                          69646
OwnerDisplayName                                                          
LastEditorUserId                                                     69646
LastEditorDisplayName                                                     
LastEditDate             

In [156]:
# now, let's drop any questions that do not have tags we are interested in

dfDSQues['Id'][dfDSQues['new_tags'] == ''].count()

5109

In [157]:
dfDSQues = dfDSQues[dfDSQues['new_tags'] != '']

In [158]:
np.shape(dfDSQues)

(1585, 23)

In [159]:
# let's now create a separate question body column, 'cleaned_question_body', that removes all the code and all HTL tags
# from the question body
# We might use the cleaned_question_body as a feature in one of the similarity models
def clean_ques_body_text(body):
    tmp_body=BeautifulSoup(body,"lxml")
    for code in tmp_body('code'):
        code.replace_with('')
    tstr=''
    for string in tmp_body.stripped_strings:
        tstr = tstr + ' ' + string
    return tstr

dfDSQues['cleaned_question_body']=''

sttime=time.time()    
dfDSQues['cleaned_question_body'] = dfDSQues['Body'].map(clean_ques_body_text)
print('question body cleaned in',(time.time()-sttime)/60,'minutes')    

C:\Users\jeffb\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\jeffb\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


question body cleaned in 0.1575226863225301 minutes


In [160]:
dfDSQues['Body'].head()

53    <p>I have a excel data with time stamp format ...
56    <p>I am learning <a href="https://en.wikipedia...
65    <p>I have trained an ANN model for a regressio...
84    <p>I'm getting confused as how to proceed for ...
90    <p>I am learning <a href="https://en.wikipedia...
Name: Body, dtype: object

In [161]:
dfDSQues['cleaned_question_body'].head()

53     I have a excel data with time stamp format li...
56     I am learning SVD by following this MIT cours...
65     I have trained an ANN model for a regression ...
84     I'm getting confused as how to proceed for th...
90     I am learning SVD by following this MIT cours...
Name: cleaned_question_body, dtype: object

In [162]:
print('questions with cleaned_question_body == Nan',len(dfDSQues['cleaned_question_body'][dfDSQues['cleaned_question_body'].isna()]))

questions with cleaned_question_body == Nan 0


In [163]:
print('questions with cleaned_question_body == "" ',len(dfDSQues['cleaned_question_body'][dfDSQues['cleaned_question_body'] == '']))

questions with cleaned_question_body == ""  3


In [164]:
# write the dataframe to a tsv
dfDSQues.to_csv("data/stackoverflow/DataSciQues_parsed.tsv", sep='\t', index=False)

In [165]:
# write the dataframe to a csv
dfDSQues.to_csv("data/stackoverflow/DataSciQues_parsed.csv",index=False)

In [21]:
# USE THIS CODE TO LOAD THE DATA SCIENCE QUESTIONS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfDSQues = pd.read_csv("data/stackoverflow/DataSciQues_parsed.tsv", sep='\t', keep_default_na=False)

In [6]:
dfDSQues

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,...,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,new_tags,cleaned_question_body
0,54374,1,54456,,2019-06-24 09:33:19,,1,590,<p>I have a excel data with time stamp format ...,76265,...,2019-08-06 15:33:14,Changing Timestamp format for Date-Time in Exc...,<python><time-series><excel>,1,1,,,,python,I have a excel data with time stamp format li...
1,54377,1,54378,,2019-06-24 09:53:27,,1,112,"<p>I am learning <a href=""https://en.wikipedia...",74719,...,2019-06-24 21:49:49,"is there a way to normalize [-3,1] to ${\begin...",<machine-learning><python><data-mining><numpy>...,1,0,2,,,python,I am learning SVD by following this MIT cours...
2,54386,1,54389,,2019-06-24 11:55:07,,0,53,<p>I have trained an ANN model for a regressio...,67537,...,2019-06-24 13:03:43,Predict using a saved regression model,<python><neural-network><deep-learning><keras>,1,3,,,,python,I have trained an ANN model for a regression ...
3,54407,1,54419,,2019-06-24 16:57:57,,0,49,<p>I'm getting confused as how to proceed for ...,76590,...,2019-06-25 02:38:11,How to predict number of orders for coming wee...,<visualization><machine-learning-model><data-s...,1,2,,,,visualization,I'm getting confused as how to proceed for th...
4,54413,1,54418,,2019-06-24 22:11:19,,1,125,"<p>I am learning <a href=""https://en.wikipedia...",74719,...,2019-06-25 02:48:41,Why does np.linalg.eig produce an opposite-sig...,<machine-learning><python><data-mining><numpy>...,1,0,0,,,python,I am learning SVD by following this MIT cours...
5,54436,1,54505,,2019-06-25 07:29:41,,0,727,"<pre class=""lang-py prettyprint-override""><cod...",76521,...,2019-06-26 00:25:12,ValueError: pos_label=1 is not a valid label: ...,<machine-learning><python><data><data-analysis...,1,0,,,,python,My training data contains the categorical fea...
6,19,1,37,,2014-05-14 03:56:20,,86,13211,<p>Lots of people use the term <em>big data</e...,84,...,2018-05-01 13:04:43,How big is big data?,<bigdata><scalability><efficiency><performance>,12,5,25,,,scala,Lots of people use the term big data in a rat...
7,116,1,121,,2014-05-17 09:16:18,,25,2818,<p>I have a database from my Facebook applicat...,173,...,2016-04-28 06:18:44,Machine learning techniques for estimating use...,<machine-learning><dimensionality-reduction><p...,6,3,12,,,python,I have a database from my Facebook applicatio...
8,23969,1,23998,,2017-10-22 07:36:15,,13,44412,<p>I'm looking to solve the following problem:...,21254,...,2019-09-02 13:55:46,Sentence similarity prediction,<python><nlp><scikit-learn><similarity><text>,5,3,12,,,python,I'm looking to solve the following problem: I...
9,23990,1,23992,,2017-10-23 02:37:45,,4,426,"<p>I am studying the blog: <a href=""http://www...",35597,...,2017-10-31 04:10:06,Understand clearly the figure: Illustration of...,<python><deep-learning><nlp><sentiment-analysi...,1,0,3,,,python,I am studying the blog: Understanding Convolu...


In [167]:
np.shape(dfDSAns)

(22666, 22)

In [168]:
dfDSAns

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,54303,2,,54237,2019-06-22 18:41:15,,1,,<p>You say you use linear regression. There ar...,71442,...,,,2019-06-22 18:41:15,,,,0,,,
1,54304,2,,54293,2019-06-22 19:06:46,,0,,<p>There are many points that you should check...,71675,...,,2019-06-23 16:04:51,2019-06-23 16:04:51,,,,5,,,
5,54314,2,,54281,2019-06-23 01:49:40,,1,,"<p>First, the remark from the wikipedia articl...",55122,...,,,2019-06-23 01:49:40,,,,5,,,
10,54319,2,,53478,2019-06-23 05:23:02,,1,,<p>As We should not remove any data ... we can...,76129,...,,,2019-06-23 05:23:02,,,,0,,,
14,54323,2,,54263,2019-06-23 09:18:46,,1,,"<p>We have </p>\n\n<p><span class=""math-contai...",38892,...,,,2019-06-23 09:18:46,,,,0,,,
15,54324,2,,54292,2019-06-23 09:25:55,,1,,"<p>Usually the <a href=""https://en.wikipedia.o...",34269,...,,,2019-06-23 09:25:55,,,,0,,,
16,54326,2,,54313,2019-06-23 10:14:40,,2,,"<blockquote>\n <p>Again, the entropy equation...",76511,...,,,2019-06-23 10:14:40,,,,1,,,
18,54328,2,,54327,2019-06-23 11:15:52,,0,,<p>Generally we track loss on validation set d...,75738,...,,,2019-06-23 11:15:52,,,,0,,,
19,54329,2,,54273,2019-06-23 11:18:15,,0,,<p>Did you try keeping a separate test set (on...,66788,...,,,2019-06-23 11:18:15,,,,0,,,
21,54331,2,,54320,2019-06-23 11:41:47,,0,,<p>This might be happening because of high lea...,75738,...,,,2019-06-23 11:41:47,,,,0,,,


In [170]:
tmpid

,tempqid
53,54374
56,54377
65,54386
84,54407
90,54413
109,54436
122,19
205,116
230,23969
241,23990


In [180]:
 pd.to_numeric(dfDSAns['ParentId'], downcast='integer')

0        54237
1        54293
5        54281
10       53478
14       54263
15       54292
16       54313
18       54327
19       54273
21       54320
25       54337
28       44800
29       54343
32       26009
33       54346
38       54352
39       54351
40       54341
42       53995
43       46624
46       54364
51       54369
52       54341
55       54296
57       54377
62       54296
64       53478
68       54386
70       54390
71       54390
         ...  
43600    14152
43602    14167
43604     9480
43605    14165
43606    14172
43609    14179
43610    14181
43614    14187
43615    14184
43616    14184
43618    14198
43619    14161
43620    14184
43623    14203
43625    14203
43626    14202
43628    14208
43629    14208
43630    13629
43631    14203
43633    14216
43635    14220
43638    13587
43639    11693
43641    14226
43643    14112
43645    14231
43647    14187
43648    11699
43649    14223
Name: ParentId, Length: 22666, dtype: int32

In [181]:
# let's drop any answers that do not have corresponding questions in our questions dataframe
tmpid=pd.DataFrame(dfDSQues['Id']) # temporary dataframe to hold ids from questions for merge operation
tmpid.columns=['tempqid']
dfDSAns=pd.merge(dfDSAns, tmpid, how='inner',left_on=pd.to_numeric(dfDSAns['ParentId'], downcast='integer'),right_on='tempqid')
del dfDSAns['tempqid']
del tmpid

In [183]:
np.shape(dfDSAns)

(2410, 22)

In [184]:
# check the answer body of each answer and extract the images references into a separate column as a list
def extract_image_references(body):
    tmp_body=BeautifulSoup(body,"lxml")
    timgs=[]
    for img in tmp_body('img'):
        timgs.append(img)
    return timgs

sttime=time.time()    

dfDSAns['images_list']=''

dfDSAns['images_list'] = dfDSAns['Body'].map(extract_image_references)
print('images finished in',(time.time()-sttime)/60,'minutes')    

images finished in 0.018283510208129884 minutes


In [185]:
# check the answer body of each answer and extract the code snippets into a separate column as a list
def extract_code_snippets(body):
    tmp_body=BeautifulSoup(body,"lxml")
    tcode_snips=[]
    for code in tmp_body('code'):
        tcode_snips.append(code)
    return tcode_snips

sttime=time.time()    

dfDSAns['code_snippets']=''

dfDSAns['code_snippets'] = dfDSAns['Body'].map(extract_code_snippets)
print('code snippets finished in',(time.time()-sttime)/60,'minutes')    

code snippets finished in 0.0185501495997111 minutes


In [187]:
# create a separate 'cleaned_body' column from the answer 'body' column with all code and all html tags removed
def clean_body_text(body):
    tmp_body=BeautifulSoup(body,"lxml")
    for code in tmp_body('code'):
        code.replace_with('')
    tstr=''
    for string in tmp_body.stripped_strings:
        tstr = tstr + ' ' + string
    return tstr

dfDSAns['cleaned_body']=''

sttime=time.time()    
dfDSAns['cleaned_body'] = dfDSAns['Body'].map(clean_body_text)
print('body cleaned in',(time.time()-sttime)/60,'minutes')  

body cleaned in 0.01733266512552897 minutes


In [188]:
dfDSAns['Body'].head()

0    <p>As We should not remove any data ... we can...
1    <p>Consider using the <a href="https://en.wiki...
2    <p>You could take an <a href="https://en.wikip...
3    <p>Did you try keeping a separate test set (on...
4    <p>From the curves you are showing <strong>yes...
Name: Body, dtype: object

In [189]:
dfDSAns['cleaned_body'].head()

0     As We should not remove any data ... we can u...
1     Consider using the Earth Mover's Distance (i....
2     You could take an Information Theory approach...
3     Did you try keeping a separate test set (on t...
4     From the curves you are showing yes . Over-fi...
Name: cleaned_body, dtype: object

In [190]:
dfDSAns['images_list'].head()

0    []
1    []
2    []
3    []
4    []
Name: images_list, dtype: object

In [191]:
dfDSAns['code_snippets'].head()

0    [<code> import numpy as np    
 import pandas ...
1                                                   []
2    [<code>&gt;&gt;&gt; from scipy.stats import en...
3                                                   []
4                                                   []
Name: code_snippets, dtype: object

In [192]:
# add an image count column to the answers so that we can filter on answers with images
def countimages(image_list):
    if image_list == []:
        return 0
    else:
        return len(image_list)
    
dfDSAns['images_count']=0
dfDSAns['images_count'] = dfDSAns['images_list'].map(countimages)

In [193]:
dfDSAns['images_count'][dfDSAns['images_count'] > 0].head()

18    2
33    1
45    1
54    1
56    1
Name: images_count, dtype: int64

In [195]:
dfDSAns['Id'][dfDSAns['images_count'] > 0].count()

197

In [196]:
# write the dataframe to a tsv
dfDSAns.to_csv("data/stackoverflow/DataSciAns_parsed.tsv", sep='\t', index=False)

In [22]:
# USE THIS CODE TO LOAD THE DATA SCIENCE ANSWERS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfDSAns = pd.read_csv("data/stackoverflow/DataSciAns_parsed.tsv", sep='\t', keep_default_na=False)

In [23]:
dfDSAns

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,...,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,images_list,code_snippets,cleaned_body,images_count
0,54319,2,,53478,2019-06-23 05:23:02,,1,,<p>As We should not remove any data ... we can...,76129,...,,,0,,,,[],[<code> import numpy as np \r\n import pand...,As We should not remove any data ... we can u...,0
1,54385,2,,53478,2019-06-24 11:54:17,,1,,"<p>Consider using the <a href=""https://en.wiki...",47371,...,,,0,,,,[],[],Consider using the Earth Mover's Distance (i....,0
2,54256,2,,53478,2019-06-21 21:55:54,,5,,"<p>You could take an <a href=""https://en.wikip...",1330,...,,,2,,,,[],[<code>&gt;&gt;&gt; from scipy.stats import en...,You could take an Information Theory approach...,0
3,54329,2,,54273,2019-06-23 11:18:15,,0,,<p>Did you try keeping a separate test set (on...,66788,...,,,0,,,,[],[],Did you try keeping a separate test set (on t...,0
4,54276,2,,54273,2019-06-22 08:10:13,,4,,<p>From the curves you are showing <strong>yes...,34269,...,,,6,,,,[],[],From the curves you are showing yes . Over-fi...,0
5,54298,2,,54273,2019-06-22 17:01:05,,-4,,<p><strong>Over-fitting happens when validatio...,71675,...,,,0,,,,[],[],Over-fitting happens when validation accuracy...,0
6,54378,2,,54377,2019-06-24 09:56:38,,6,,"<p>You have already computed that, but you've ...",28175,...,,,0,,,,[],"[<code>result = np.linalg.norm(v1,ord=2,axis=1...","You have already computed that, but you've no...",0
7,54389,2,,54386,2019-06-24 13:03:43,,1,,<p>Use <code>sklearn.preprocessing.OneHotEncod...,69822,...,,,0,,,,[],[<code>sklearn.preprocessing.OneHotEncoder</co...,Use for example and transfer the one-hot enco...,0
8,54418,2,,54413,2019-06-25 02:38:07,,3,,<p>Any scalar multiple of an eigenvector is al...,55122,...,,,4,,,,[],[],Any scalar multiple of an eigenvector is also...,0
9,54419,2,,54407,2019-06-25 02:38:11,,0,,<p>If you treat this problem as a time-series ...,76540,...,,,1,,,,[],[],If you treat this problem as a time-series on...,0


In [24]:
# let's check to see if there duplicate ids in the Stack Overflow and Data Science Stack Exchange datasets before
# combining them
tsoquesids=pd.DataFrame(dfPostQuestionsFiltered['id'])
tsoansids=pd.DataFrame(dfPostAnswersFiltered['id'])
tdsquesids=pd.DataFrame(dfDSQues['Id'])
tdsansids=pd.DataFrame(dfDSAns['Id'])
tmergeques=pd.merge(tsoquesids, tdsquesids, how='inner', left_on='id', right_on='Id')
tmergeans=pd.merge(tsoansids, tdsansids, how='inner', left_on='id', right_on='Id')

In [25]:
tmergeques

,id,Id
0,58649,58649


In [26]:
tmergeans

,id,Id
0,24943,24943
1,52921,52921
2,57408,57408
3,43239,43239
4,2517,2517
5,6087,6087
6,51804,51804
7,14525,14525
8,26700,26700
9,33886,33886


In [18]:
print('length of dataframes without dropping duplicate Ids -- SO ques:',len(dfPostQuestionsFiltered), 'DS ques', len(dfDSQues),'SO ans:',len(dfPostAnswersFiltered), 'DS ans', len(dfDSAns) )

length of dataframes without dropping duplicate Ids -- SO ques: 173698 DS ques 1585 SO ans: 261052 DS ans 2410


In [28]:
# dropping questions with duplicate ids from the Stack Overflow and data science question sets
# also dropping answers that refer back to those duplicate questions
# note: we do not drop answers that happen to have duplicate ids as the risk of getting bad return results
# in our models is minimal for duplicate answers, espcially given the small number of duplicate answer ids
qidstodrop=tmergeques['Id'].values.tolist()
print('duplicate question Ids to be dropped:',qidstodrop)

duplicate question Ids to be dropped: [58649]


In [33]:
for dupqid in qidstodrop:
    
    SOanswithdupqids=dfPostAnswersFiltered['id'][dfPostAnswersFiltered['parent_id']==dupqid].index
    print('dropping',len(SOanswithdupqids),'answers from SO answers that have question parent id =',dupqid)
    dfPostAnswersFiltered.drop(labels=SOanswithdupqids,inplace=True)
    
    DSanswithdupqids=dfDSAns['Id'][dfDSAns['ParentId']==dupqid].index
    print('dropping',len(DSanswithdupqids),'answers from DS answers that have question parent id =',dupqid)
    dfDSAns.drop(labels=DSanswithdupqids,inplace=True)

    print('dropping quesion from SO that has question =',dupqid)
    dfPostQuestionsFiltered.drop(labels=dfPostQuestionsFiltered[dfPostQuestionsFiltered['id']==dupqid].index,inplace=True)

    print('dropping quesion from DS that has question =',dupqid)
    dfDSQues.drop(labels=dfDSQues[dfDSQues['Id']==dupqid].index,inplace=True)
    

dropping 7 answers from SO answers that have question parent id = 58649
dropping 1 answers from DS answers that have question parent id = 58649
dropping quesion from SO that has question = 58649
dropping quesion from DS that has question = 58649


In [34]:
print('length of dataframes after dropping duplicate question Ids -- SO ques:',len(dfPostQuestionsFiltered), 'DS ques', len(dfDSQues),'SO ans:',len(dfPostAnswersFiltered), 'DS ans', len(dfDSAns) )

length of dataframes after dropping duplicate question Ids -- SO ques: 173697 DS ques 1584 SO ans: 261045 DS ans 2409


In [35]:
dfPostAnswersFiltered.columns

Index(['id', 'title', 'body', 'accepted_answer_id', 'answer_count',
       'comment_count', 'community_owned_date', 'creation_date',
       'favorite_count', 'last_activity_date', 'last_edit_date',
       'last_editor_display_name', 'last_editor_user_id', 'owner_display_name',
       'owner_user_id', 'parent_id', 'post_type_id', 'score', 'tags',
       'view_count', 'images_list', 'code_snippets', 'cleaned_body',
       'images_count'],
      dtype='object')

In [36]:
dfDSAns.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'DeletionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
       'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName',
       'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
       'CommentCount', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate',
       'images_list', 'code_snippets', 'cleaned_body', 'images_count'],
      dtype='object')

In [37]:
# drop the columns in the data science answers that are not also in the Stack Overflow answers before merging both
del dfDSAns['DeletionDate']
del dfDSAns['ClosedDate']

In [38]:
# change the column names in data science answers dataset to match those in the Stack Overflow answers dataset
dfDSAns.columns = ['id', 'post_type_id', 'accepted_answer_id', 'parent_id', 'creation_date', 'score', 
                   'view_count', 'body', 'owner_user_id', 'owner_display_name', 'last_editor_user_id', 
                   'last_editor_display_name', 'last_edit_date', 'last_activity_date', 'title', 'tags',
                   'answer_count', 'comment_count', 'favorite_count', 'community_owned_date', 
                   'images_list', 'code_snippets', 'cleaned_body', 'images_count']

In [39]:
dfCombinedAns=dfPostAnswersFiltered.append(dfDSAns,ignore_index=True)

C:\Users\jeffb\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [40]:
dfCombinedAns

,accepted_answer_id,answer_count,body,cleaned_body,code_snippets,comment_count,community_owned_date,creation_date,favorite_count,id,...,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,title,view_count
0,,,<p>The correct syntax is this one:</p>\r\r\n\r...,The correct syntax is this one: It is working...,[<code>plugin install com.graphaware.es/graph-...,0,,2017-06-19 13:43:45.580000+00:00,,44632276,...,,,,5222773.0,43895941,2,0,,,
1,,,<p>You can download the plugin manually from t...,You can download the plugin manually from the...,[],13,,2017-05-12 09:28:09.857000+00:00,,43934125,...,,,,2662355.0,43895941,2,1,,,
2,,,<p>You can define the lines from the dataset w...,You can define the lines from the dataset wit...,"[<code>mapping.get(i, ""anchor""),\r\r\n</code>]",0,,2017-06-19 13:51:27.790000+00:00,,44632434,...,,,,3951036.0,44621561,2,0,,,
3,,,<p>The way matplotlib is working is that you h...,The way matplotlib is working is that you hav...,"[<code>def error_plot(ax, title, x_data, y_dat...",0,,2017-06-19 14:47:46.830000+00:00,,44633649,...,,,,4124317.0,44633422,2,0,,,
4,,,"<p>I have figured out the answer, which is to ...","I have figured out the answer, which is to us...","[<code>%sql select t1.time, t1.value,coalesce(...",0,,2017-06-19 14:48:42.900000+00:00,,44633675,...,,,,8050923.0,44624707,2,0,,,
5,,,<p>To further a_guest answer - To enable see t...,To further a_guest answer - To enable see the...,[],0,,2017-06-19 15:56:06.457000+00:00,,44635048,...,,,,1704227.0,44632018,2,0,,,
6,,,<p>The problem is that the Python process fini...,The problem is that the Python process finish...,"[<code>for</code>, <code>-i</code>, <code>pyth...",3,,2017-06-19 14:12:13.690000+00:00,,44632901,...,,,,3767239.0,44632018,2,2,,,
7,,,<p>You can keep the window open by creating a ...,You can keep the window open by creating a at...,"[<code>QApplication</code>, <code>exec_()</cod...",0,,2018-04-23 15:59:00.010000+00:00,,49985313,...,,,,6605826.0,44632018,2,1,,,
8,,,<p>Since you've mentioned that the transformat...,Since you've mentioned that the transformatio...,[<code>lambda=3.79\r\r\nm2=lm(resp^((lambda-1)...,0,,2017-06-19 17:10:08.570000+00:00,,44636285,...,,,,8060278.0,18506201,2,0,,,
9,,,"<pre><code>plotexpl &lt;- seq(1,4,length.out=1...",I won't discuss the statistical issues here (...,"[<code>plotexpl &lt;- seq(1,4,length.out=10)\r...",3,,2013-08-29 09:24:47.800000+00:00,,18507004,...,,,,1412059.0,18506201,2,2,,,


In [41]:
np.shape(dfPostAnswersFiltered)

(261045, 24)

In [42]:
np.shape(dfDSAns)

(2409, 24)

In [43]:
np.shape(dfCombinedAns)

(263454, 24)

In [44]:
dfPostQuestionsFiltered.columns

Index(['id', 'title', 'body', 'accepted_answer_id', 'answer_count',
       'comment_count', 'community_owned_date', 'creation_date',
       'favorite_count', 'last_activity_date', 'last_edit_date',
       'last_editor_display_name', 'last_editor_user_id', 'owner_display_name',
       'owner_user_id', 'parent_id', 'post_type_id', 'score', 'tags',
       'view_count', 'new_tags', 'cleaned_question_body'],
      dtype='object')

In [45]:
dfDSQues.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'DeletionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
       'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName',
       'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
       'CommentCount', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate',
       'new_tags', 'cleaned_question_body'],
      dtype='object')

In [46]:
# drop the columns in the data science answers that are not also in the Stack Overflow answers before merging both
del dfDSQues['DeletionDate']
del dfDSQues['ClosedDate']

In [48]:
# change the column names in data science answers dataset to match those in the Stack Overflow answers dataset
dfDSQues.columns = ['id', 'post_type_id', 'accepted_answer_id', 'parent_id', 'creation_date', 'score', 
                   'view_count', 'body', 'owner_user_id', 'owner_display_name', 'last_editor_user_id', 
                   'last_editor_display_name', 'last_edit_date', 'last_activity_date', 'title', 'tags',
                   'answer_count', 'comment_count', 'favorite_count', 'community_owned_date', 
                   'new_tags', 'cleaned_question_body']

In [49]:
dfCombinedQues=dfPostQuestionsFiltered.append(dfDSQues,ignore_index=True)

C:\Users\jeffb\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [50]:
dfCombinedQues

,accepted_answer_id,answer_count,body,cleaned_question_body,comment_count,community_owned_date,creation_date,favorite_count,id,last_activity_date,...,last_editor_user_id,new_tags,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,title,view_count
0,56196332.0,1,<p>I have configured and set up a fully functi...,I have configured and set up a fully function...,0,,2019-05-18 04:35:44.290000+00:00,,56195999,2019-05-18 05:45:46.330000+00:00,...,7916354.0,graph,,7916354.0,,1,2,reactjserror-handlinggraphqlapollographql-js,Apollo client doesn't display an error message,256
1,56212940.0,1,<p>I want to use a mutation in an event handle...,I want to use a mutation in an event handler....,0,,2019-05-19 22:47:04.330000+00:00,,56212612,2019-05-19 23:57:38.440000+00:00,...,,graph,,10688712.0,,1,0,reactjsgraphqlreact-hooksmutation,Calling a hook (useMutation) inside an event h...,261
2,56206915.0,2,<p>I have useQuery and useMutation from react-...,I have useQuery and useMutation from react-ap...,0,,2019-05-19 04:52:19.730000+00:00,,56204854,2019-07-15 12:45:21.457000+00:00,...,,graph,,10116367.0,,1,0,javascriptreactjsgraphqlreact-apolloreact-hooks,How do I chain useQuery and useMutation in Gra...,262
3,56197449.0,1,<p>I'm coding in R and I have the following da...,I'm coding in R and I have the following data...,0,,2019-05-17 15:01:14.930000+00:00,,56189061,2019-05-18 08:58:44.763000+00:00,...,,plot plotly,,9537627.0,,1,0,rplotly,Customize lines based on specific criteria wit...,11
4,56203343.0,1,<p>I am graphing the internet connection in my...,I am graphing the internet connection in my a...,0,,2019-05-18 22:17:19.290000+00:00,,56203260,2019-05-18 22:32:10.980000+00:00,...,,python graph,,8598201.0,,1,0,pythonlistgraphsumiteration,How do I sum a list with its previous entries ...,11
5,56220931.0,1,<p>I have the following script</p>\r\r\n\r\r\n...,I have the following script Expected Behavior...,0,,2019-05-20 09:43:44.660000+00:00,,56218250,2019-05-20 12:22:29.497000+00:00,...,,matplotlib plot,,2749397.0,,1,0,matplotlib,Modify y data in a line,11
6,56275655.0,1,"<p>I'm using Google's <a href=""https://develop...",I'm using Google's geochart . I'm using the n...,0,,2019-05-23 12:33:33.063000+00:00,,56275333,2019-05-23 12:51:20.197000+00:00,...,,chart visualization,,930151.0,,1,0,chartsgoogle-visualizationgeo,Display Google Geochart in log scale,11
7,55973074.0,1,<p>I am trying to create create a calculator(f...,I am trying to create create a calculator(for...,0,,2019-05-03 15:22:50.823000+00:00,,55972829,2019-05-03 15:36:57.783000+00:00,...,,plot plotly,,10067658.0,,1,0,javascriptplotly.js,Data displayed upon submission of a form is vi...,12
8,56095454.0,1,<p>I am trying to draw a line from google char...,I am trying to draw a line from google chart ...,0,,2019-05-12 01:01:47.690000+00:00,,56095434,2019-05-12 01:16:52.103000+00:00,...,,visualization,,11375519.0,,1,0,phpmysqlgoogle-visualization,Every row given must be either null or an arra...,12
9,56260575.0,1,"<p>For a bar or column chart in Highcharts, is...","For a bar or column chart in Highcharts, is t...",0,,2019-05-22 15:33:03.250000+00:00,,56260378,2019-05-22 15:44:21.623000+00:00,...,,chart highcharts,,614263.0,,1,0,chartshighchartsstatistics,Make Highcharts pick a smart y-axis min,13


In [51]:
np.shape(dfPostQuestionsFiltered)

(173697, 22)

In [52]:
np.shape(dfDSQues)

(1584, 22)

In [53]:
np.shape(dfCombinedQues)

(175281, 22)

In [54]:
# write the combined questions dataframe to a tsv
dfCombinedQues.to_csv("data/stackoverflow/SODSQues_parsed.tsv", sep='\t', index=False)

In [55]:
# write the combined questions dataframe to a csv
dfCombinedQues.to_csv("data/stackoverflow/SODSQues_parsed.csv", index=False)

In [22]:
# USE THIS CODE TO LOAD THE SO + DATA SCIENCE QUESTIONS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfCombinedQues = pd.read_csv("data/stackoverflow/SODSQues_parsed.tsv", sep='\t', keep_default_na=False)

In [56]:
# write the combined answers dataframe to a tsv
dfCombinedAns.to_csv("data/stackoverflow/SODSAns_parsed.tsv", sep='\t', index=False)

In [22]:
# USE THIS CODE TO LOAD THE SO + DATA SCIENCE ANSWERS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfCombinedAns = pd.read_csv("data/stackoverflow/SODSAns_parsed.tsv", sep='\t', keep_default_na=False)